In [1]:
%load_ext nb_black

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

import pandas as pd
import numpy as np
import seaborn as sns
import altair as alt
import json

<IPython.core.display.Javascript object>

In [2]:
data = pd.read_csv("./data/archon_arcana_weekly_20210619.csv", thousands=",")

data = data.iloc[1:, :]  # drop first line (not a full week)
data

,1. day: 31000,Weekly,Daily,Total,Sun 10AM,COTA Wk,COTA Day,COTA Tot,AOA Wk,AOA Day,AOA Tot,WC Wk,WC Day,WC Tot,MM Wk,MM Day,MM Tot,DT Wk,DT Day,DT Tot
1,NaN,82656,11808,158016,Nov 25,82656,11808,158016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,60717,8674,218733,Dec 2,60717,8674,218733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,49293,7042,268026,Dec 9,49293,7042,268026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,43869,6267,311895,Dec 16,43869,6267,311895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,46203,6600,358098,Dec 23,46203,6600,358098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,May 14: USA,17247,2464,2308213,May 16,2593,370,1172876,1738.0,248.0,491615.0,927.0,132.0,362155.0,1253.0,179.0,236483.0,10736.0,1534.0,45084.0
131,May 18: DT DoK live,24620,3517,2332833,May 23,2006,287,1174882,1529.0,218.0,493144.0,950.0,136.0,363105.0,1617.0,231.0,238100.0,18518.0,2645.0,63602.0
132,NaN,14035,2005,2346868,May 30,1642,235,1176524,1441.0,206.0,494585.0,1146.0,164.0,364251.0,1850.0,264.0,239950.0,7956.0,1137.0,71558.0
133,NaN,12244,1749,2359112,Jun 6,1604,229,1178128,1864.0,266.0,496449.0,1497.0,214.0,365748.0,1782.0,255.0,241732.0,5497.0,785.0,77055.0


<IPython.core.display.Javascript object>

In [3]:
import io
import requests

url = "https://raw.githubusercontent.com/4dcu-be/GitHub-Actions-KeyForge/main/data/keyforge_decks.tsv"
req = requests.get(url).content
new_data = pd.read_csv(io.StringIO(req.decode("utf-8")), sep="\t")
new_data["date"] = pd.to_datetime(new_data["date"])
new_data

,date,count
0,2021-06-01 19:22:33.563763,2350740
1,2021-06-01 19:35:55.387635,2350751
2,2021-06-01 21:19:03.615943,2351072
3,2021-06-01 22:46:43.408636,2351254
4,2021-06-01 23:32:58.745276,2351346
...,...,...
1743,2021-08-16 05:55:33.874900,2431748
1744,2021-08-16 06:56:25.989287,2431780
1745,2021-08-16 07:55:31.626072,2431801
1746,2021-08-16 08:55:24.186680,2431812


<IPython.core.display.Javascript object>

In [4]:
new_data.dtypes

date     datetime64[ns]
count             int64
dtype: object

<IPython.core.display.Javascript object>

In [5]:
new_data["date_only"] = new_data["date"].dt.strftime("%Y-%m-%d")
new_data["dow"] = new_data["date"].dt.strftime("%a")
new_data["day"] = new_data["date"].dt.strftime("%b %d")
grouped_df = (
    new_data.groupby(["date_only"])
    .agg(
        count=pd.NamedAgg("count", max),
        dow=pd.NamedAgg("dow", "first"),
        day=pd.NamedAgg("day", "first"),
    )
    .reset_index()
)
grouped_df = grouped_df[grouped_df.dow == "Sun"]
grouped_df = grouped_df[grouped_df["count"] > np.max(data.Total)]
grouped_df = grouped_df.iloc[1:, :]  # drop first line overlap with historic data
grouped_df

,date_only,count,dow,day
19,2021-06-20,2377188,Sun,Jun 20
26,2021-06-27,2384362,Sun,Jun 27
33,2021-07-04,2391312,Sun,Jul 04
40,2021-07-11,2399203,Sun,Jul 11
47,2021-07-18,2406672,Sun,Jul 18
54,2021-07-25,2413273,Sun,Jul 25
61,2021-08-01,2419440,Sun,Aug 01
68,2021-08-08,2425048,Sun,Aug 08
75,2021-08-15,2431634,Sun,Aug 15


<IPython.core.display.Javascript object>

In [6]:
data_subset = data[["Total", "Sun 10AM"]].rename(
    columns={"Total": "count", "Sun 10AM": "day"}
)
output_df = (
    pd.concat([data_subset, grouped_df[["count", "day"]]])
    .reset_index()
    .drop(columns=["index"])
)
output_df["week"] = output_df.index + 1
output_df

,count,day,week
0,158016,Nov 25,1
1,218733,Dec 2,2
2,268026,Dec 9,3
3,311895,Dec 16,4
4,358098,Dec 23,5
...,...,...,...
138,2406672,Jul 18,139
139,2413273,Jul 25,140
140,2419440,Aug 01,141
141,2425048,Aug 08,142


<IPython.core.display.Javascript object>

In [7]:
output_df.to_csv("./data/update_simple.tab", sep="\t")

<IPython.core.display.Javascript object>